In [70]:
import torch
import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.data import DataLoader


dataset = torch.load('meshgraphnets_miniset5traj_vis.pt')

/tmp/ipykernel_179291/2625543928.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load('meshgraphnets_miniset5traj_vis.pt')


In [71]:
print(dataset[0].x.shape, dataset[0].num_nodes)

torch.Size([1923, 11]) 1923


In [73]:
graph = dataset[0]
# initialize 2x2 sizing field for each node in the graph
sizing_field = torch.tensor([[[600,0],[0,600]]] * graph.num_nodes)

print(graph.edge_index)

for e in range(graph.num_edges):
    i, j = graph.edge_index[:,e][0], graph.edge_index[:,e][1]
    S_i, S_j = sizing_field[i], sizing_field[j]
    u_ij = graph.mesh_pos[i] - graph.mesh_pos[j]
    S_ij = (S_i + S_j) / 2
    if u_ij.T @ S_ij @ u_ij > 1:
        neighbors_i = graph.edge_index[1, graph.edge_index[0] == i]
        neighbors_j = graph.edge_index[1, graph.edge_index[0] == j]
        neighbors = np.intersect1d(neighbors_i, neighbors_j)

        # remove edge
        print(graph.edge_index.shape)
        graph.edge_index = torch.cat([graph.edge_index[:, :e], 
                                      graph.edge_index[:, e+1:]], 
                                      dim=1)
        
        new_node = graph.x.shape[0]
        new_node_x = (graph.x[i] + graph.x[j]) / 2
        new_node_mesh_pos = (graph.mesh_pos[i] + graph.mesh_pos[j]) / 2
        new_node_p = (graph.p[i] + graph.p[j]) / 2

        graph.num_nodes += 1
        graph.x = torch.cat([graph.x, new_node_x.unsqueeze(0)])
        graph.mesh_pos = torch.cat([graph.mesh_pos, new_node_mesh_pos.unsqueeze(0)])
        graph.p = torch.cat([graph.p, new_node_p.unsqueeze(0)])

        new_edges = torch.tensor([[i, new_node],
                                  [j, new_node]]
                                 + [[n, new_node] for n in neighbors]).T
        print(new_edges)
        graph.edge_index = torch.cat([graph.edge_index, new_edges], dim=1)
        graph.num_edges += 2 + len(neighbors)
        
print('done')
        

tensor([[   1,    4,    7,  ..., 1473, 1486, 1937],
        [   0,    3,    6,  ..., 1952, 1952, 1952]])
torch.Size([2, 11171])
torch.Size([2, 11170])
tensor([[  46,   44,   47],
        [1953, 1953, 1953]])
torch.Size([2, 11173])
torch.Size([2, 11172])
tensor([[  27,   14,   28],
        [1954, 1954, 1954]])
torch.Size([2, 11175])
torch.Size([2, 11174])
tensor([[  43,   41,   42],
        [1955, 1955, 1955]])
torch.Size([2, 11177])
torch.Size([2, 11176])
tensor([[  44,   46,   47, 1953],
        [1956, 1956, 1956, 1956]])
torch.Size([2, 11180])
torch.Size([2, 11179])
tensor([[  14,   27,   28, 1954],
        [1957, 1957, 1957, 1957]])
torch.Size([2, 11183])
torch.Size([2, 11182])
tensor([[  41,   43,   42, 1955],
        [1958, 1958, 1958, 1958]])


IndexError: index 1953 is out of bounds for dimension 0 with size 1953